In [38]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [39]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [40]:
loader=WebBaseLoader("https://www.ibm.com/think/topics/langchain")
loader

In [41]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://www.ibm.com/think/topics/langchain', 'title': 'What Is LangChain? | IBM', 'description': 'LangChain is an open source orchestration framework for the development of applications using large language models (LLMs), like chatbots and virtual agents.\u202f', 'language': 'en'}, page_content="\n\n\n\n\n\n\n\n\n\nWhat Is LangChain? | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                   \n\n\n\n  \n    What is LangChain?\n\n\n\n\n\n\n    \n\n\n                               \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAI Agents\n\n\n\nWelcome\n\n\n\n\n\nCaret right\n\nIntroduction\n\n\n\n\nOverview\n\n\n\n\nAI agents vs AI assistants\n\n\n\n\nAgentic AI\n\n\n\n\nAgentic AI vs generative AI\n\n\n\n\n\nCaret right\n\nAI agent 

In [42]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [43]:
documents

[Document(metadata={'source': 'https://www.ibm.com/think/topics/langchain', 'title': 'What Is LangChain? | IBM', 'description': 'LangChain is an open source orchestration framework for the development of applications using large language models (LLMs), like chatbots and virtual agents.\u202f', 'language': 'en'}, page_content='What Is LangChain? | IBM\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                   \n\n\n\n  \n    What is LangChain?\n\n\n\n\n\n\n    \n\n\n                               \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAI Agents\n\n\n\nWelcome\n\n\n\n\n\nCaret right\n\nIntroduction\n\n\n\n\nOverview\n\n\n\n\nAI agents vs AI assistants\n\n\n\n\nAgentic AI\n\n\n\n\nAgentic AI vs generative AI\n\n\n\n\n\nCaret right\n\nAI agent development\n\n\n\n\

In [44]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()


In [45]:
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embeddings)

In [46]:
vectorstoredb


In [47]:
## Query From a vector db
query="LangChain serves as a generic interface for nearly any LLM, providing a centralized development environment to build LLM applications and integrate them with external data sources and software workflows."
result=vectorstoredb.similarity_search(query)
result[0].page_content

'LangChain serves as a generic interface for nearly any LLM, providing a centralized development environment to build LLM applications and integrate them with external data sources and software workflows. LangChain’s module-based approach allows developers and data scientists to dynamically compare different prompts and even different foundation models with minimal need to rewrite code. This modular environment also allows for programs that use multiple LLMs: for example, an application that uses one LLM to interpret user queries and another LLM to author a response.'

In [48]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [53]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x3118605c0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x311861d60>, root_client=<openai.OpenAI object at 0x17ca99880>, root_async_client=<openai.AsyncOpenAI object at 0x17eb02450>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_docume

In [54]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"",
    "context":[Document(page_content="LangChain serves as a generic interface for nearly any LLM, providing a centralized development environment to build LLM applications and integrate them with external data sources and software workflows. LangChain’s module-based approach allows developers and data scientists to dynamically compare different prompts and even different foundation models with minimal need to rewrite code.")]
})

'How does LangChain benefit developers and data scientists when working with LLM applications?\n\nLangChain benefits developers and data scientists by offering a centralized development environment that simplifies the process of building LLM applications. Its module-based approach allows users to dynamically compare different prompts and foundation models, minimizing the need to rewrite code. This flexibility and efficiency make it easier to integrate LLMs with external data sources and software workflows.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [55]:
### Input--->Retriever--->vectorstoredb
vectorstoredb

In [56]:
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [57]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x17eb03500>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | ChatOpe

In [59]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"LangChain serves as a generic interface for nearly any LLM, providing a centralized development environment to build LLM applications and integrate them with external data sources and software workflows. "})
response['answer']

"LangChain works by providing a development environment that streamlines the programming of LLM applications through the use of abstraction. This involves representing complex processes as named components that encapsulate all their constituent steps. LangChain acts as a library of abstractions in Python and JavaScript, offering modular components like functions and object classes, which can be chained together to create generative AI applications. This modular, abstracted approach allows developers to dynamically compare different prompts and foundation models without rewriting much code. LangChain supports custom LLM wrappers and integrates with external models, such as those from IBM's watsonx, to build applications tailored to specific needs. While this abstracted approach might limit detailed customization by expert programmers, it simplifies the process for both specialists and newcomers, enabling quick experimentation and prototyping of LLM-based applications."

In [60]:
response

{'input': 'LangChain serves as a generic interface for nearly any LLM, providing a centralized development environment to build LLM applications and integrate them with external data sources and software workflows. ',
 'context': [Document(id='cbe9cbc9-3cc2-4dc3-9735-408fe66e87ba', metadata={'source': 'https://www.ibm.com/think/topics/langchain', 'title': 'What Is LangChain? | IBM', 'description': 'LangChain is an open source orchestration framework for the development of applications using large language models (LLMs), like chatbots and virtual agents.\u202f', 'language': 'en'}, page_content='LangChain serves as a generic interface for nearly any LLM, providing a centralized development environment to build LLM applications and integrate them with external data sources and software workflows. LangChain’s module-based approach allows developers and data scientists to dynamically compare different prompts and even different foundation models with minimal need to rewrite code. This modul

In [61]:
response['context']

[Document(id='cbe9cbc9-3cc2-4dc3-9735-408fe66e87ba', metadata={'source': 'https://www.ibm.com/think/topics/langchain', 'title': 'What Is LangChain? | IBM', 'description': 'LangChain is an open source orchestration framework for the development of applications using large language models (LLMs), like chatbots and virtual agents.\u202f', 'language': 'en'}, page_content='LangChain serves as a generic interface for nearly any LLM, providing a centralized development environment to build LLM applications and integrate them with external data sources and software workflows. LangChain’s module-based approach allows developers and data scientists to dynamically compare different prompts and even different foundation models with minimal need to rewrite code. This modular environment also allows for programs that use multiple LLMs: for example, an application that uses one LLM to interpret user queries and another LLM to author a response.'),
 Document(id='a81fe579-d493-4e90-8e9d-8c40c18709f0', 